# SemViT vs. BPG Comparison (Colab Ready)

This notebook generates a performance comparison chart between:
1.  **Your Trained SemViT Models** (Mixed, LEO, GEO, AWGN)
2.  **BPG + Capacity** (Theoretical Upper Bound for BPG)
3.  **BPG + LDPC** (Simulated Adaptive BPG Performance)

### Instructions
1.  **Run Setup**: Run the first code cell to install dependencies (`sionna`, `tensorflow-compression`, etc.).
2.  **Upload Repository**: Ensure the `models/` and `utils/` folders from the repo are present.
3.  **Upload Weights**: Upload your trained model weights to a `weights/` folder (or update the paths below).
4.  **Run All Cells**: The notebook will evaluate your models and plot them against the hardcoded BPG baselines.


In [ ]:
# --- SETUP: Install Dependencies ---

# IF YOU CLONED THE REPO: Un-comment the next line to install strict versions from requirements.txt
# !pip install -r requirements.txt

# OTHERWISE: Run this to install critical libraries (Relaxed versions for Colab compatibility)
!pip install sionna==0.14.0 tensorflow-compression tensorflow-addons mitsuba==3.2.1

# Note: If 'tensorflow-compression' fails to install, the code handles it gracefully (using standard layers).
# You can ignore the error if you see "No matching distribution found".

# --- AUTO-NAVIGATION ---
# If you cloned the repo, we'll try to enter the directory automatically.
import os
repo_name = 'Semmantic-Communication-Geo-Leo-Channels' # Adjust if your folder name differs
if os.path.exists(repo_name):
    os.chdir(repo_name)
    print(f"Changed directory to {os.getcwd()}")
elif os.path.exists('/content/' + repo_name):
    os.chdir('/content/' + repo_name)
    print(f"Changed directory to {os.getcwd()}")

In [ ]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Ensure we can import from the repo structure
import sys
if not os.path.exists('models'):
    print("WARNING: 'models' directory not found. Please clone the repo or upload files.")

from models.model import SemViT
from utils.datasets import dataset_generator

# Disable eager execution for performance if needed, though Keras 3 might prefer it.
# For this repo's TF version (likely <2.16 given the code style), disabling might be safer for loops.
tf.config.run_functions_eagerly(False)

In [ ]:
# --- CONFIGURATION ---

# 1. Test Conditions
SNR_RANGE = [0, 2, 5, 7, 10, 12, 15]  # The SNRs we have BPG data for
TEST_CHANNEL = 'AWGN'                 # Channel to test User Models on
DATA_SIZE = 512                       # 1/6 Bandwidth Ratio (512 / 32*32*3)

# 2. User Models to Evaluate
# UPDATE THESE PATHS to match where you uploaded your files in Colab
MODELS_TO_TEST = {
    "Mixed (LEO+GEO)": "weights/experiment_mixed_sat_138.weights.h5",
    "LEO Only":        "weights/experiment_leo_sat_128.weights.h5",
    "GEO Only":        "weights/experiment_geo_sat_34.weights.h5",
    "AWGN (Baseline)": "weights/CCVVCC_512_10dB_599.weights.h5"
}

# 3. BPG Baselines (Extracted from bpg-ldpc.ipynb)
# Bandwidth Ratio = 1/6

# Theoretical Limit: BPG compressed to Channel Capacity size
BPG_CAPACITY_DATA = {
    "PSNR": [24.13, 25.65, 28.24, 29.72, 31.59, 32.83, 34.54],
    "SSIM": [0.82,  0.87,  0.92,  0.94,  0.96,  0.97,  0.98]
}

# Practical Baseline: BPG + LDPC (Adaptive Modulation/Coding)
# Best performing MCS chosen for each SNR from the simulation logs
BPG_LDPC_DATA = {
    "PSNR": [21.66, 24.13, 25.43, 27.86, 29.73, 30.49, 32.83],
    "SSIM": [0.72,  0.82,  0.86,  0.92,  0.94,  0.95,  0.97]
}

# 4. Model Architecture (Must match training)
BLOCK_TYPES = ['C', 'C', 'V', 'V', 'C', 'C']
FILTERS = [256, 256, 256, 256, 256, 256]
NUM_BLOCKS = [1, 1, 3, 3, 1, 1]
HAS_GDN = True

In [ ]:
# --- DATASET LOADER ---
print("Preparing Dataset...")
def normalize_img(x, y):
    return (tf.cast(x, tf.float32) / 255.0, tf.cast(x, tf.float32) / 255.0)

# Tries to load CIFAR-10. If not found, it downloads it.
try:
    test_ds_raw = dataset_generator('./dataset/CIFAR10/test/', shuffle=False)
except:
    print("Dataset not found locally. Downloading CIFAR-10...")
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    # Create a tf.data.Dataset from the numpy arrays
    test_ds_raw = tf.data.Dataset.from_tensor_slices((x_test, y_test))
    test_ds_raw = test_ds_raw.batch(1) # generator usually returns batched

# Take a subset for quicker evaluation (e.g., 200 images)
# Remove .take(200) to evaluate on the whole set (slower)
test_ds = test_ds_raw.map(normalize_img).take(200).prefetch(tf.data.AUTOTUNE)

In [ ]:
# --- EVALUATION FUNCTION ---
def evaluate_model(weights_path, snr_db):
    print(f"  Loading {weights_path} for SNR {snr_db}dB...")
    
    # Initialize Model
    model = SemViT(
        block_types=BLOCK_TYPES,
        filters=FILTERS,
        num_blocks=NUM_BLOCKS,
        has_gdn=HAS_GDN,
        num_symbols=DATA_SIZE,
        snrdB=snr_db,
        channel=TEST_CHANNEL 
    )
    model.compile(optimizer='adam', loss='mse')
    # Build passing dummy input
    model(tf.zeros((1, 32, 32, 3))) 
    
    # Load Weights
    if not os.path.exists(weights_path):
        print(f"    [ERROR] File not found: {weights_path}")
        return None, None
    
    try:
        model.load_weights(weights_path, skip_mismatch=True)
    except Exception as e:
        print(f"    [ERROR] Failed to load weights: {e}")
        return None, None

    # Run Inference
    psnr_sum = 0.0
    ssim_sum = 0.0
    count = 0
    
    for x, y in test_ds:
        recon = model(x, training=False)
        # Calculate Metrics
        batch_psnr = tf.reduce_mean(tf.image.psnr(x, recon, max_val=1.0))
        batch_ssim = tf.reduce_mean(tf.image.ssim(x, recon, max_val=1.0))
        
        psnr_sum += float(batch_psnr)
        ssim_sum += float(batch_ssim)
        count += 1
    
    tf.keras.backend.clear_session()
    return (psnr_sum / count), (ssim_sum / count)

In [ ]:
# --- MAIN LOOP ---
results = {name: {"PSNR": [], "SSIM": []} for name in MODELS_TO_TEST}

for model_name, path in MODELS_TO_TEST.items():
    print(f"Evaluating {model_name}...")
    current_snr_psnr = []
    current_snr_ssim = []
    
    for snr in SNR_RANGE:
        p, s = evaluate_model(path, snr)
        if p is None: 
            p, s = 0.0, 0.0 # Handle missing weights gracefully for plot
        current_snr_psnr.append(p)
        current_snr_ssim.append(s)
        print(f"    SNR={snr}: PSNR={p:.2f}, SSIM={s:.3f}")
    
    results[model_name]["PSNR"] = current_snr_psnr
    results[model_name]["SSIM"] = current_snr_ssim

In [ ]:
# --- PLOTTING ---
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Colors for User Models
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple']

# 1. PSNR Plot
ax = axes[0]
# -- Baselines --
ax.plot(SNR_RANGE, BPG_CAPACITY_DATA["PSNR"], label="BPG + Capacity", 
        color='black', linestyle='--', linewidth=2, marker='^')
ax.plot(SNR_RANGE, BPG_LDPC_DATA["PSNR"], label="BPG + LDPC", 
        color='gray', linestyle='-.', linewidth=2, marker='v')
# -- User Models --
for i, (name, metrics) in enumerate(results.items()):
    ax.plot(SNR_RANGE, metrics["PSNR"], label=name, marker='o', linewidth=2.5, color=colors[i % len(colors)])

ax.set_title("PSNR Comparison", fontsize=14)
ax.set_xlabel("SNR (dB)", fontsize=12)
ax.set_ylabel("PSNR (dB)", fontsize=12)
ax.grid(True, alpha=0.3)
ax.legend(fontsize=10)

# 2. SSIM Plot
ax = axes[1]
# -- Baselines --
ax.plot(SNR_RANGE, BPG_CAPACITY_DATA["SSIM"], label="BPG + Capacity", 
        color='black', linestyle='--', linewidth=2, marker='^')
ax.plot(SNR_RANGE, BPG_LDPC_DATA["SSIM"], label="BPG + LDPC", 
        color='gray', linestyle='-.', linewidth=2, marker='v')
# -- User Models --
for i, (name, metrics) in enumerate(results.items()):
    ax.plot(SNR_RANGE, metrics["SSIM"], label=name, marker='o', linewidth=2.5, color=colors[i % len(colors)])

ax.set_title("SSIM Comparison", fontsize=14)
ax.set_xlabel("SNR (dB)", fontsize=12)
ax.set_ylabel("SSIM", fontsize=12)
ax.grid(True, alpha=0.3)
ax.legend(fontsize=10)

plt.tight_layout()
plt.savefig("colab_comparison_chart.png", dpi=300)
plt.show()